##**Train**

In [ ]:
import time
from torchvision import models
from torchsummary import summary

In [ ]:
train_json = "/content/drive/MyDrive/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/json/part_A_train.json"
test_json = "/content/drive/MyDrive/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/json/part_A_test.json"
task = "A"
pre = None
best_prec1 = 1e6
original_lr = 1e-7
lr = 1e-7
batch_size = 1
momentum = 0.95
decay = 5*1e-4
start_epoch = 0
epochs = 10
steps = [-1,1,100,150]
scales = [1,1,1,1]
workers = 4
seed = time.time()
print_freq = 30

In [ ]:
from torch.functional import Tensor
from torchvision.transforms.transforms import Resize
import sys
import os
import warnings
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import argparse
import json
import cv2
import time

def train(train_list, model, criterion, optimizer, epoch):
    
    losses = AverageMeter()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    
    
    train_loader = torch.utils.data.DataLoader(
        listDataset(train_list,
                       shuffle=True,
                       transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Resize(768),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                       ]), 
                       target_transform=transforms.Compose([                                 
                        transforms.Resize(768),
                       ]),
                       train=True, 
                       seen=model.seen,
                       batch_size=batch_size,
                       num_workers=workers),
        batch_size=batch_size)
    print('epoch %d, processed %d samples, lr %.10f' % (epoch, epoch * len(train_loader.dataset), lr))
    
    model.train()
    end = time.time()
    
    for i,(img, target)in enumerate(train_loader):
        data_time.update(time.time() - end)
        
        img = img.to(device)
        img = Variable(img)
 
        output = model(img)
        #output = nn.functional.interpolate(output, scale_factor=2, mode="bicubic")
      
        target = target.type(torch.FloatTensor).unsqueeze(0).to(device)
        target = Variable(target)
        
        loss = criterion(output, target)
        losses.update(loss.item(), img.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  .format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))
    return losses.avg
    
def validate(val_list, model, criterion):
    print ('begin test')
    test_loader = torch.utils.data.DataLoader(
    listDataset(val_list,
                   shuffle=False,
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Resize(768),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                       ]), 
                   target_transform=transforms.Compose([                                 
                        transforms.Resize(768),
                       ]),
                   train=False),
    batch_size=batch_size)    
    model.eval()
    mae = 0
    mse = 0
    with torch.no_grad():
      for i,(img, target) in enumerate(test_loader):
        img = img.to(device)
        img = Variable(img)
        output = model(img)
        mse += criterion(output, target.type(torch.FloatTensor).to(device))
        mae += abs(output.data.sum()-target.sum().type(torch.FloatTensor).to(device))
    mse = mse/len(test_loader)  
    mae = mae/len(test_loader)    
    print(f'* MAE {mae:.3f} , MSE {mse:.3f}')

    return mae, mse 
        
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = original_lr
    
    for i in range(len(steps)):
        
        scale = scales[i] if i < len(scales) else 1
        
        
        if epoch >= steps[i]:
            lr = lr * scale
            if epoch == steps[i]:
                break
        else:
            break
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count          

###**Execution time, Parameters and FLOPs**

In [ ]:
pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

####**CUDA execution time**

In [ ]:
model = CSRNet().cuda()
inputs = torch.randn(5, 3, 768, 768).cuda()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
volta_scudnn_winograd_128x128_ldg1_ldg4_relu_tile148...         0.00%       0.000us         0.00%       0.000us       0.000us     289.609ms        55.14%     289.609ms      32.179ms             9  
                   volta_scudnn_128x64_relu_small_nn_v1         0.00%       0.000us         0.00%       0.000us       0.000us     165.651ms        31.54%     165.651ms      23.664ms             7  
void at::

In [ ]:
print(prof.key_averages())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  cudaStreamIsCapturing         0.00%      25.000us         0.00%      25.000us       0.641us       0.000us         0.00%       0.000us       0.000us            39  
                                             cudaMalloc         0.21%      18.484ms         0.21%      18.484ms     637.379us       0.000us         0.00%       0.000us       0.000us            29  
         

In [ ]:
print(prof.key_averages())

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         cudaGetDeviceCount         0.12%     484.000us         0.12%     484.000us     484.000us             1  
    cudaGetDeviceProperties         0.03%     124.000us         0.03%     124.000us     124.000us             1  
      cudaDeviceSynchronize        99.85%     408.747ms        99.85%     408.747ms     408.747ms             1  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 409.355ms



In [ ]:
with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

In [ ]:
print(prof.key_averages())

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
    cudaDeviceSynchronize       100.00%      36.000us       100.00%      36.000us      36.000us             1  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 36.000us



In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
    cudaDeviceSynchronize       100.00%      36.000us       100.00%      36.000us      36.000us             1  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 36.000us



####**CPU execution time**

In [ ]:
model = CSRNet()
inputs = torch.randn(5, 3, 768, 768)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.00%     305.000us         0.00%     305.000us       9.242us     663.75 Mb     663.75 Mb            33  
    aten::max_pool2d_with_indices         5.09%     326.428ms         5.09%     326.428ms     108.809ms     189.00 Mb     189.00 Mb             3  
                    aten::resize_         0.00%      19.000us         0.00%      19.000us      19.000us      36.00 Kb      36.00 Kb             1  
                     aten::conv2d         0.00%     156.000us        93.75%        6.017s     353.932ms     663.

In [ ]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.00%     156.000us        93.75%        6.017s     353.932ms     663.79 Mb           0 b            17  
                aten::convolution         0.01%     478.000us        93.75%        6.017s     353.922ms     663.79 Mb           0 b            17  
               aten::_convolution         0.01%     372.000us        93.74%        6.016s     353.894ms     663.79 Mb           0 b            17  
         aten::mkldnn_convolution        93.72%        6.015s        93.73%        6.016s     375.969ms     663.

In [ ]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         0.54%     193.505ms       100.00%       35.550s       35.550s             1  
                     aten::conv2d         0.00%     150.000us        94.04%       33.432s        1.967s            17  
                aten::convolution         0.00%     531.000us        94.04%       33.432s        1.967s            17  
               aten::_convolution         0.00%     352.000us        94.04%       33.431s        1.967s            17  
         aten::mkldnn_convolution        94.02%       33.423s        94.03%       33.427s        2.089s            16  
                 aten::max_pool2d       

####**FLOPs and params**

In [ ]:
#用ptflops

In [ ]:
pip install ptflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ptflops import get_model_complexity_info
model = CSRNet()
flops, params = get_model_complexity_info(model, ( 3, 768, 768), as_strings=True, print_per_layer_stat=True)
print('Flops:  ' + flops)
print('Params: ' + params)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSRNet(
  16.26 M, 100.000% Params, 244.01 GMac, 100.000% MACs, 
  (frontend): Sequential(
    7.64 M, 46.947% Params, 164.47 GMac, 67.404% MACs, 
    (0): Conv2d(1.79 k, 0.011% Params, 1.06 GMac, 0.433% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 37.75 MMac, 0.015% MACs, inplace=True)
    (2): Conv2d(36.93 k, 0.227% Params, 21.78 GMac, 8.926% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 37.75 MMac, 0.015% MACs, inplace=True)
    (4): MaxPool2d(0, 0.000% Params, 37.75 MMac, 0.015% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(73.86 k, 0.454% Params, 10.89 GMac, 4.463% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(0, 0.000% Params, 18.87 MMac, 0.008% MACs, inplace=True)
    (7): Conv2d(147.58 k, 0.907% Params, 21.76 GMac, 8.919% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(0,

In [ ]:
#用fvcore

In [ ]:
pip install fvcore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 54.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=676deba8d11da1565fb67f3a3449f3d5aa3a0e3ae8af1a7ad78e6c917baceaee
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=079154c0b77c101ba8ecbbc5ba0d1b3d90574da85d83aa223e2c6d3798d07957
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from fvcore.nn import FlopCountAnalysis, parameter_count_table
model = CSRNet()
tensor = (torch.rand(1, 3, 768, 768),)
# 分析FLOPs
flops = FlopCountAnalysis(model, tensor)
print("FLOPs: ", flops.total())

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FLOPs:  243593183232


In [ ]:
# 分析parameters
print(parameter_count_table(model))

| name                  | #elements or shape   |
|:----------------------|:---------------------|
| model                 | 16.3M                |
|  frontend             |  7.6M                |
|   frontend.0          |   1.8K               |
|    frontend.0.weight  |    (64, 3, 3, 3)     |
|    frontend.0.bias    |    (64,)             |
|   frontend.2          |   36.9K              |
|    frontend.2.weight  |    (64, 64, 3, 3)    |
|    frontend.2.bias    |    (64,)             |
|   frontend.5          |   73.9K              |
|    frontend.5.weight  |    (128, 64, 3, 3)   |
|    frontend.5.bias    |    (128,)            |
|   frontend.7          |   0.1M               |
|    frontend.7.weight  |    (128, 128, 3, 3)  |
|    frontend.7.bias    |    (128,)            |
|   frontend.10         |   0.3M               |
|    frontend.10.weight |    (256, 128, 3, 3)  |
|    frontend.10.bias   |    (256,)            |
|   frontend.12         |   0.6M               |
|    frontend.12.wei

In [ ]:
#用torchstat庫

In [ ]:
pip install torchstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchstat import stat
model = CSRNet()
stat(model, (3, 768, 768))

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


        module name  input shape output shape      params memory(MB)               MAdd              Flops   MemRead(B)  MemWrite(B) duration[%]     MemR+W(B)
0        frontend.0    3 768 768   64 768 768      1792.0     144.00    2,038,431,744.0    1,056,964,608.0    7085056.0  150994944.0       1.10%  1.580800e+08
1        frontend.1   64 768 768   64 768 768         0.0     144.00       37,748,736.0       37,748,736.0  150994944.0  150994944.0       0.23%  3.019899e+08
2        frontend.2   64 768 768   64 768 768     36928.0     144.00   43,486,543,872.0   21,781,020,672.0  151142656.0  150994944.0       8.46%  3.021376e+08
3        frontend.3   64 768 768   64 768 768         0.0     144.00       37,748,736.0       37,748,736.0  150994944.0  150994944.0       0.19%  3.019899e+08
4        frontend.4   64 768 768   64 384 384         0.0      36.00       28,311,552.0       37,748,736.0  150994944.0   37748736.0       2.58%  1.887437e+08
5        frontend.5   64 384 384  128 384 384 

####**macs and params**

In [ ]:
from thop import profile
model = CSRNet()
inputs = torch.randn(1, 3, 768, 768)
macs, params = profile(model, inputs=(inputs, ))

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.


In [ ]:
from thop import profile
model = CSRNet()
inputs = torch.randn(1, 3, 768, 768)
macs, params = profile(model, inputs=(inputs, ))

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.


In [ ]:
macs

243593183232.0

In [ ]:
params

16263489.0

In [ ]:
from thop import clever_format #科學記號算法
macs, params = clever_format([macs, params], "%.3f")

In [ ]:
macs

'243.593G'

In [ ]:
params

'16.263M'

####**macs and params (per layer + sum)**

In [ ]:
pip install ptflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ptflops: filename=ptflops-0.6.9-py3-none-any.whl size=11712 sha256=a848f37787b788c215b2e050834ac09ed52f32685f10107070d22622eed5ff4a
  Stored in directory: /root/.cache/pip/wheels/c8/71/2f/92426c1ef33fb2e275b533878d8378f91c7f26846d9669019c
Successfully built ptflops


In [ ]:
import torchvision.models as models
import torch
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
  model = CSRNet()
  macs, params = get_model_complexity_info(model, (3, 768, 768), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSRNet(
  16.26 M, 100.000% Params, 244.01 GMac, 100.000% MACs, 
  (frontend): Sequential(
    7.64 M, 46.947% Params, 164.47 GMac, 67.404% MACs, 
    (0): Conv2d(1.79 k, 0.011% Params, 1.06 GMac, 0.433% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0, 0.000% Params, 37.75 MMac, 0.015% MACs, inplace=True)
    (2): Conv2d(36.93 k, 0.227% Params, 21.78 GMac, 8.926% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0, 0.000% Params, 37.75 MMac, 0.015% MACs, inplace=True)
    (4): MaxPool2d(0, 0.000% Params, 37.75 MMac, 0.015% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(73.86 k, 0.454% Params, 10.89 GMac, 4.463% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(0, 0.000% Params, 18.87 MMac, 0.008% MACs, inplace=True)
    (7): Conv2d(147.58 k, 0.907% Params, 21.76 GMac, 8.919% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(0,

###**writer**

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter('/content/drive/MyDrive/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data')

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

###**training**

In [ ]:
#batch_size=1, epochs = 10

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False).to(device)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for epoch in range(start_epoch, epochs):     
  adjust_learning_rate(optimizer, epoch)
        
  train_loss = train(train_list, model, criterion, optimizer, epoch)
  writer.add_scalar('Train/loss', train_loss, epoch)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, epoch)
  writer.add_scalar('Validaiton/MSE', mse_val, epoch)

  # writer.add_histogram('model.bias', model.bias, epoch)
  # writer.add_histogram('model.weight', model.weight, epoch)
  # writer.add_histogram('model.weight.grad', model.weight.grad, epoch)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)
  torch.save(model.state_dict(),'CSRNet.pt')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 1, 1, 96, 96])) that is different to the input size (torch.Size([1, 1, 96, 96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: [0][0/1200]	Time 7.647 (7.647)	Data 0.818 (0.818)	Loss 160.5666 (160.5666)	
Epoch: [0][30/1200]	Time 0.904 (0.959)	Data 0.799 (0.634)	Loss 128.4258 (249.9519)	
Epoch: [0][60/1200]	Time 0.761 (0.848)	Data 0.669 (0.627)	Loss 17.8781 (188.8093)	
Epoch: [0][90/1200]	Time 0.278 (0.777)	Data 0.025 (0.586)	Loss 22.8210 (202.3100)	
Epoch: [0][120/1200]	Time 1.005 (0.747)	Data 0.912 (0.570)	Loss 36.1149 (180.7707)	
Epoch: [0][150/1200]	Time 0.699 (0.733)	Data 0.604 (0.564)	Loss 229.4871 (175.1565)	
Epoch: [0][180/1200]	Time 0.827 (0.707)	Data 0.730 (0.539)	Loss 15.1810 (213.0221)	
Epoch: [0][210/1200]	Time 0.803 (0.693)	Data 0.709 (0.529)	Loss 43.8914 (200.9069)	
Epoch: [0][240/1200]	Time 0.784 (0.671)	Data 0.688 (0.504)	Loss 107.4975 (206.8680)	
Epoch: [0][270/1200]	Time 0.682 (0.647)	Data 0.584 (0.476)	Loss 58.2443 (197.1103)	
Epoch: [0][300/1200]	Time 0.301 (0.631)	Data 0.032 (0.457)	Loss 4.7098 (184.2461)	
Epoch: [0][330/1200]	Time 0.289 (0.618)	Data 0.028 (0.443)	Loss 88.3614 (182.2

In [ ]:
#batch_size=1, epochs = 10

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for epoch in range(start_epoch, epochs):     
  adjust_learning_rate(optimizer, epoch)
        
  train_loss = train(train_list, model, criterion, optimizer, epoch)
  writer.add_scalar('Train/loss', train_loss, epoch)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, epoch)
  writer.add_scalar('Validaiton/MSE', mse_val, epoch)

  # writer.add_histogram('model.bias', model.bias, epoch)
  # writer.add_histogram('model.weight', model.weight, epoch)
  # writer.add_histogram('model.weight.grad', model.weight.grad, epoch)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)
  torch.save(model.state_dict(),'CSRNet.pt')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 2, 1, 96, 96])) that is different to the input size (torch.Size([2, 1, 96, 96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: [0][0/600]	Time 9.298 (9.298)	Data 2.422 (2.422)	Loss 209.7955 (209.7955)	
Epoch: [0][30/600]	Time 1.981 (2.780)	Data 1.772 (2.305)	Loss 51.3667 (313.9127)	
Epoch: [0][60/600]	Time 2.976 (2.570)	Data 2.678 (2.201)	Loss 42.5507 (408.6763)	
Epoch: [0][90/600]	Time 1.383 (2.367)	Data 1.175 (2.029)	Loss 79.8393 (400.6017)	
Epoch: [0][120/600]	Time 0.612 (2.186)	Data 0.052 (1.851)	Loss 257.1412 (369.1639)	
Epoch: [0][150/600]	Time 1.426 (2.059)	Data 1.219 (1.725)	Loss 25.1952 (368.4917)	
Epoch: [0][180/600]	Time 1.398 (1.944)	Data 1.191 (1.610)	Loss 233.4849 (385.4198)	
Epoch: [0][210/600]	Time 0.597 (1.845)	Data 0.051 (1.507)	Loss 31.5115 (401.5128)	
Epoch: [0][240/600]	Time 0.608 (1.724)	Data 0.048 (1.370)	Loss 19.0383 (387.1889)	
Epoch: [0][270/600]	Time 0.604 (1.650)	Data 0.054 (1.288)	Loss 161.4347 (386.1187)	
Epoch: [0][300/600]	Time 1.812 (1.567)	Data 1.602 (1.191)	Loss 58.0986 (368.6510)	
Epoch: [0][330/600]	Time 0.598 (1.502)	Data 0.055 (1.117)	Loss 393.8622 (373.0839)	
Epoc

In [ ]:
#batch_size=3, epochs = 10

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for epoch in range(start_epoch, epochs):     
  adjust_learning_rate(optimizer, epoch)
        
  train_loss = train(train_list, model, criterion, optimizer, epoch)
  writer.add_scalar('Train/loss', train_loss, epoch)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, epoch)
  writer.add_scalar('Validaiton/MSE', mse_val, epoch)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 3, 1, 96, 96])) that is different to the input size (torch.Size([3, 1, 96, 96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: [0][0/400]	Time 0.419 (0.419)	Data 0.074 (0.074)	Loss 450.0032 (450.0032)	
Epoch: [0][30/400]	Time 0.891 (0.861)	Data 0.070 (0.075)	Loss 168.5710 (728.7531)	
Epoch: [0][60/400]	Time 0.867 (0.870)	Data 0.073 (0.075)	Loss 1946.7571 (643.4725)	
Epoch: [0][90/400]	Time 0.859 (0.866)	Data 0.074 (0.075)	Loss 709.7125 (646.9154)	
Epoch: [0][120/400]	Time 0.870 (0.865)	Data 0.069 (0.076)	Loss 2167.3120 (603.9414)	
Epoch: [0][150/400]	Time 0.871 (0.866)	Data 0.085 (0.077)	Loss 168.5392 (589.0890)	
Epoch: [0][180/400]	Time 0.866 (0.866)	Data 0.085 (0.077)	Loss 156.2336 (619.4141)	
Epoch: [0][210/400]	Time 0.867 (0.866)	Data 0.056 (0.077)	Loss 44.4137 (629.2695)	
Epoch: [0][240/400]	Time 0.868 (0.866)	Data 0.066 (0.077)	Loss 105.2403 (630.2514)	
Epoch: [0][270/400]	Time 0.866 (0.866)	Data 0.093 (0.077)	Loss 152.2068 (631.4106)	
Epoch: [0][300/400]	Time 0.874 (0.866)	Data 0.079 (0.077)	Loss 299.1494 (613.2259)	
Epoch: [0][330/400]	Time 0.865 (0.866)	Data 0.075 (0.077)	Loss 439.8683 (603.884

In [ ]:
#batch_size=3, epochs = 15

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for i in range(start_epoch, epochs):     
     
  adjust_learning_rate(optimizer, i)
        
  train_loss = train(train_list, model, criterion, optimizer, i)
  writer.add_scalar('Train/loss', train_loss, i)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, i)
  writer.add_scalar('Validaiton/MSE', mse_val, i)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'i': i + 1,
          # 'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 3, 1, 96, 96])) that is different to the input size (torch.Size([3, 1, 96, 96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: [0][0/400]	Time 1.261 (1.261)	Data 0.866 (0.866)	Loss 2687.6086 (2687.6086)	
Epoch: [0][30/400]	Time 1.414 (1.036)	Data 1.115 (0.695)	Loss 308.9787 (763.7194)	
Epoch: [0][60/400]	Time 0.878 (0.993)	Data 0.292 (0.611)	Loss 325.8677 (666.4696)	
Epoch: [0][90/400]	Time 0.874 (0.976)	Data 0.069 (0.552)	Loss 1016.2009 (632.8001)	
Epoch: [0][120/400]	Time 0.870 (0.954)	Data 0.526 (0.487)	Loss 442.7638 (630.2451)	
Epoch: [0][150/400]	Time 0.868 (0.938)	Data 0.328 (0.431)	Loss 100.1702 (600.2356)	
Epoch: [0][180/400]	Time 0.878 (0.929)	Data 0.086 (0.394)	Loss 191.6761 (650.5561)	
Epoch: [0][210/400]	Time 0.869 (0.922)	Data 0.468 (0.371)	Loss 572.9136 (624.7142)	
Epoch: [0][240/400]	Time 0.871 (0.917)	Data 0.061 (0.340)	Loss 80.5137 (635.2691)	
Epoch: [0][270/400]	Time 0.876 (0.911)	Data 0.063 (0.316)	Loss 98.3986 (632.6325)	
Epoch: [0][300/400]	Time 0.871 (0.907)	Data 0.065 (0.296)	Loss 39.9298 (639.2606)	
Epoch: [0][330/400]	Time 0.869 (0.904)	Data 0.091 (0.278)	Loss 238.2113 (621.0006

In [ ]:
#batch_size=5, epochs = 30

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for i in range(1200):
#for i in range(start_epoch, epochs):     
     
  adjust_learning_rate(optimizer, i)
        
  train_loss = train(train_list, model, criterion, optimizer, i)
  writer.add_scalar('Train/loss', train_loss, i)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, i)
  writer.add_scalar('Validaiton/MSE', mse_val, i)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'i': i + 1,
          # 'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch 0, processed 0 samples, lr 0.0000001000


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 5, 1, 96, 96])) that is different to the input size (torch.Size([5, 1, 96, 96])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: [0][0/240]	Time 12.410 (12.410)	Data 3.714 (3.714)	Loss 872.3701 (872.3701)	
Epoch: [0][30/240]	Time 3.416 (3.180)	Data 2.839 (2.403)	Loss 1371.8308 (1276.7616)	
Epoch: [0][60/240]	Time 1.511 (2.637)	Data 0.590 (1.927)	Loss 865.1785 (1019.1992)	
Epoch: [0][90/240]	Time 1.482 (2.341)	Data 0.134 (1.584)	Loss 4102.6646 (1071.9643)	
Epoch: [0][120/240]	Time 1.490 (2.151)	Data 0.868 (1.345)	Loss 2136.6816 (1048.2500)	
Epoch: [0][150/240]	Time 1.494 (2.026)	Data 0.177 (1.168)	Loss 526.7922 (960.6864)	
Epoch: [0][180/240]	Time 1.496 (1.938)	Data 0.134 (1.010)	Loss 442.9261 (924.8119)	
Epoch: [0][210/240]	Time 1.494 (1.875)	Data 0.122 (0.887)	Loss 843.1969 (957.4155)	
begin test
* MAE 821.377 , MSE 617.714
 * best MAE 821.377 
epoch 1, processed 1200 samples, lr 0.0000001000
Epoch: [1][0/240]	Time 0.693 (0.693)	Data 0.121 (0.121)	Loss 95.3606 (95.3606)	
Epoch: [1][30/240]	Time 1.484 (1.483)	Data 0.125 (0.122)	Loss 200.0798 (988.1533)	
Epoch: [1][60/240]	Time 1.497 (1.484)	Data 0.123 (0.

In [ ]:
#epochs = 10

import random

with open(train_json, 'r') as outfile:        
    train_list = json.load(outfile)
with open(test_json, 'r') as outfile:       
    val_list = json.load(outfile)
    
torch.cuda.manual_seed(seed)
    
model = CSRNet()
    
model = model.to(device)

criterion = nn.MSELoss(size_average=False)
    
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=decay)

if pre:
  if os.path.isfile(pre):
    print("=> loading checkpoint '{}'".format(pre))
    checkpoint = torch.load(pre)
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})".format(pre, checkpoint['epoch']))
  else:
    print("=> no checkpoint found at '{}'".format(pre))


for epoch in range(start_epoch, epochs):     
  adjust_learning_rate(optimizer, epoch)
        
  train_loss = train(train_list, model, criterion, optimizer, epoch)
  writer.add_scalar('Train/loss', train_loss, epoch)
  mae_val, mse_val = validate(val_list, model, criterion)
  writer.add_scalar('Validation/MAE', mae_val, epoch)
  writer.add_scalar('Validaiton/MSE', mse_val, epoch)
        
  is_best = mae_val < best_prec1
  best_prec1 = min(mae_val, best_prec1)
  print(' * best MAE {mae:.3f} '.format(mae=best_prec1))
  save_checkpoint({
          'epoch': epoch + 1,
          'arch': pre,
          'state_dict': model.state_dict(),
          'best_prec1': best_prec1,
          'optimizer' : optimizer.state_dict(), 
          }, is_best,task)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1

epoch 0, processed 0 samples, lr 0.0000001000
Epoch: [0][0/1200]	Time 0.186 (0.186)	Data 0.024 (0.024)	Loss 21.5536 (21.5536)	
Epoch: [0][30/1200]	Time 0.295 (0.291)	Data 0.024 (0.046)	Loss 166.0893 (148.5645)	
Epoch: [0][60/1200]	Time 0.297 (0.294)	Data 0.023 (0.043)	Loss 103.3777 (238.4104)	
Epoch: [0][90/1200]	Time 0.303 (0.296)	Data 0.056 (0.041)	Loss 22.8667 (188.9219)	
Epoch: [0][120/1200]	Time 0.304 (0.298)	Data 0.030 (0.040)	Loss 616.3179 (182.8980)	
Epoch: [0][150/1200]	Time 0.308 (0.300)	Data 0.044 (0.039)	Loss 28.4209 (214.7974)	
Epoch: [0][180/1200]	Time 0.306 (0.301)	Data 0.098 (0.039)	Loss 181.6534 (191.3812)	
Epoch: [0][210/1200]	Time 0.296 (0.302)	Data 0.104 (0.040)	Loss 963.5510 (189.8652)	
Epoch: [0][240/1200]	Time 0.302 (0.302)	Data 0.031 (0.041)	Loss 50.4121 (187.1717)	
Epoch: [0][270/1200]	Time 0.304 (0.302)	Data 0.022 (0.041)	Loss 13.8582 (179.5239)	
Epoch: [0][300/1200]	Time 0.299 (0.302)	Data 0.022 (0.040)	Loss 16.3452 (190.6148)	
Epoch: [0][330/1200]	Time 0.299

##**Validate the results**

In [ ]:
#importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
import torch
%matplotlib inline

In [ ]:
#transform
from torchvision import datasets, transforms
transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Resize(768),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                       ])

In [ ]:
#defining the location of dataset
root = '/content/drive/MyDrive/ShanghaiTech_Crowd_Counting_Dataset'
part_A_train = os.path.join(root,'part_A/train_data','images')
part_A_test = os.path.join(root,'part_A/test_data','images')
part_B_train = os.path.join(root,'part_B/train_data','images')
part_B_test = os.path.join(root,'part_B/test_data','images')
path_sets = [part_A_test]

In [ ]:
#defining the image path
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
       img_paths.append(img_path)
 
#model
model = CSRNet()
 

#defining the model
model = model.to(device)
 
#loading the trained weights
checkpoint = torch.load('/content/drive/MyDrive/ShanghaiTech_Crowd_Counting_Dataset/checkpoint.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [ ]:
img= plt.imread(img_path)
k = np.zeros((img.shape[0],img.shape[1]))

In [ ]:
mae = 0
for i in range(len(img_paths)):
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))

    img[0,:,:]=img[0,:,:]-92.8207477031
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.to(device)
    #img = transform(Image.open(img_paths[i]).convert('RGB')).to(device)
    gt_file = np.load(img_paths[i].replace('.jpg','.mpy').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
print(mae/len(img))

0.0


In [ ]:
def validate(val_list, model, criterion):
  model.eval()
  mae = 0
  #mse = 0
  for(img,target) in val_list:
    img = img.to(device)
    target = torch.unqueeze(target,1).to(device)
    output = model(img)

    mae+= abs(output.data.sum()-target.data.sum())
    #mse+= pow((output.data.sum()-target.data.sum()),2)
  mae = mae/criterion
  #mse = pow((mse/dataset_num),0.5)

print('MAE = {mae:.3f}'.format(mae = mae))
#print('MAE = {mae:.3f} MSE = {mse:3f}'.format(mae =mae,mse = mse))

MAE = 0.000
